In [788]:
import findspark
findspark.init()

In [789]:
import pathlib

from pyspark.sql import (
                        functions as sf,
                        SparkSession,
                        types,
                        Window
)


In [790]:
spark = SparkSession.builder.master("local").getOrCreate()

In [791]:
CODE_GROUPS_SCHEMA = types.StructType([
    types.StructField("ICD_CODE",types.StringType(),nullable=True),
    types.StructField("GROUP",types.StringType(),nullable=True),
])

DEMOGRAPHICS_SCHEMA = types.StructType([
    types.StructField("CLINIC",types.StringType(),nullable=True),
    types.StructField("MRN",types.StringType(),nullable=True),
    types.StructField("FIRST_NAME",types.StringType(),nullable=True),
    types.StructField("LAST_NAME",types.StringType(),nullable=True),
    types.StructField("DATE_OF_BIRTH",types.DateType(),nullable=True),
])

ENCOUNTERS_SCHEMA = types.StructType([
    types.StructField("DATE",types.DateType(),nullable=True),
    types.StructField("ENC_ID",types.StringType(),nullable=True),
    types.StructField("MRN",types.StringType(),nullable=True),
    types.StructField("ICD_CODE",types.StringType(),nullable=True),
])

In [792]:
def csv_to_parquet(path):
    # Spark requires path as a str
    path = str(path)

    filename = path.split("/")[-1].split(".")[0]

    if filename.casefold() == "code_groups":
        schema = CODE_GROUPS_SCHEMA
    elif filename.casefold() == "demographics":
        schema = DEMOGRAPHICS_SCHEMA
    elif filename.casefold() == "encounters":
        schema = ENCOUNTERS_SCHEMA

    df = (
        spark.read
        .option("delimiter", "|")
        .option("header", "true")
        .schema(schema)
        .csv(path)
    )
            
    parquet_path = path.split(".")[0]+".parquet"

    df.write.mode("overwrite").parquet(parquet_path)

In [793]:
def read_csv(path):
    path = str(path)

    filename = path.split("/")[-1].split(".")[0]

    if filename.casefold() == "code_groups":
        schema = CODE_GROUPS_SCHEMA
    elif filename.casefold() == "demographics":
        schema = DEMOGRAPHICS_SCHEMA
    elif filename.casefold() == "encounters":
        schema = ENCOUNTERS_SCHEMA

    df = (
        spark.read
        .option("delimiter", "|")
        .option("header", "true")
        .schema(schema)
        .csv(path)
    )

    return df


In [794]:

# This can be rewritten to use Spark-native functions.
# This removes a dependency we don't use otherwise
def read_parquet(path):
    path = str(path)

    df = spark.read.parquet(path)
    
    return df

In [795]:
def rename_columns(sdf, name_map):
    for from_col, to_col in name_map.items():
        sdf = sdf.withColumnRenamed(from_col, to_col)
    return sdf

In [796]:
def age_bucket(age):
    if age > 100:
        return 100
    elif age > 90:
        return 90
    elif age > 80:
        return 80
    elif age > 70:
        return 70
    elif age > 60:
        return 60
    elif age > 50:
        return 50
    elif age > 40:
        return 40
    else:
        # ignore all patients under 40
        return 0

In [797]:
# change this to data_2, data_3 or data_4 to use a different dataset
data_folder = pathlib.Path('data_test')

code_groups = pathlib.Path('code_groups.csv')
demographics = data_folder.joinpath('demographics.csv')
encounters = data_folder.joinpath('encounters.csv')

for csv in [code_groups, demographics, encounters]:
    csv_to_parquet(csv)

code_groups = pathlib.Path('code_groups.parquet')
demographics = data_folder.joinpath('demographics.parquet')
encounters = data_folder.joinpath('encounters.parquet')

In [798]:
code_groups_sdf = read_parquet(code_groups)
demo_sdf = read_parquet(demographics)
enc_sdf = read_parquet(encounters)

In [799]:
# Data quality checks

def check_dupes(df):
    return (
        df.groupBy(df.columns)
        .count()
        .where(sf.col("count") > 1)
        .select(sf.coalesce(sf.sum("count"), sf.lit(0)))
        .withColumnRenamed("coalesce(sum(count), 0)", f"Duplicate Count")
        .show()
    )

def check_nulls(df):
    print("Null Counts")
    return df.select([sf.count(sf.when(sf.col(c).isNull(), c)).alias(c) for c in df.columns]
   ).show()

def run_single_df_checks(df):
    check_dupes(df)
    check_nulls(df)
    df.describe().show()

def run_all_checks():
    print("-----CODE GROUPS-----")
    run_single_df_checks(code_groups_sdf)
    print("-----DEMO-----")
    run_single_df_checks(demo_sdf)
    print("-----ENC-----")
    run_single_df_checks(enc_sdf)

run_all_checks()

-----CODE GROUPS-----


+---------------+
|Duplicate Count|
+---------------+
|              0|
+---------------+

Null Counts
+--------+-----+
|ICD_CODE|GROUP|
+--------+-----+
|       0|    0|
+--------+-----+

+-------+--------+------------------+
|summary|ICD_CODE|             GROUP|
+-------+--------+------------------+
|  count|    4995|              4995|
|   mean|    null| 5.531131131131131|
| stddev|    null|2.8903991264796347|
|    min|   A07.0|                 1|
|    max| Z98.818|                 9|
+-------+--------+------------------+

-----DEMO-----


+---------------+
|Duplicate Count|
+---------------+
|              0|
+---------------+

Null Counts
+------+---+----------+---------+-------------+
|CLINIC|MRN|FIRST_NAME|LAST_NAME|DATE_OF_BIRTH|
+------+---+----------+---------+-------------+
|     0|  0|         0|        0|            0|
+------+---+----------+---------+-------------+

+-------+------+--------------------+----------+---------+
|summary|CLINIC|                 MRN|FIRST_NAME|LAST_NAME|
+-------+------+--------------------+----------+---------+
|  count| 10000|               10000|     10000|    10000|
|   mean|  null|     5.07491028763E7|      null|     null|
| stddev|  null|2.8727825630081754E7|      null|     null|
|    min|     A|            01015674|     Aaron|   Abbott|
|    max|     C|            99998883|       Zoe|�the Kid�|
+-------+------+--------------------+----------+---------+

-----ENC-----


+---------------+
|Duplicate Count|
+---------------+
|              0|
+---------------+

Null Counts
+----+------+---+--------+
|DATE|ENC_ID|MRN|ICD_CODE|
+----+------+---+--------+
|   0|     0|  0|       0|
+----+------+---+--------+



+-------+------------------+---------+--------+
|summary|            ENC_ID|      MRN|ICD_CODE|
+-------+------------------+---------+--------+
|  count|            131447|   131447|  131447|
|   mean|4.98692226390104E7|     null|    null|
| stddev|2.87755318476725E7|     null|    null|
|    min|          10000905|A01126333|   A07.0|
|    max|          99999335|C99975199| Z98.818|
+-------+------------------+---------+--------+



In [800]:
def clean_data(df, type):
    # Pulled from https://www.johndcook.com/blog/2019/05/05/regex_icd_codes/
    N = "\d{3}\.?\d{0,2}"
    E = "E\d{3}\.?\d?"
    V = "V\d{2}\.?\d{0,2}"
    icd9_regex = "|".join([N, E, V])
    icd10_regex = "[A-TV-Z][0-9][0-9AB]\.?[0-9A-TV-Z]{0,4}"

    # Allows weird dates like 0000-99-99 but good enough for our purposes
    date_regex = "[0-9]{4}-[0-9]{2}-[0-9]{2}"
    #name_regex = r"[A-Za-z]+"

    if type == "code_groups":
        df = (df.where(
            (sf.col("ICD_CODE").isNotNull())
        &   (sf.col("GROUP").isNotNull())
        &   (sf.col("ICD_CODE").rlike(icd9_regex) | sf.col("ICD_CODE").rlike(icd10_regex))
        ))
    elif type == "demographics":
        df = (df.where(
            (sf.col("DATE_OF_BIRTH").isNotNull())
        &   (sf.col("MRN").isNotNull())
        &   (sf.col("DATE_OF_BIRTH").rlike(date_regex))
        &   (sf.col("DATE_OF_BIRTH") > sf.date_sub(sf.current_date(), (120 * 365)))
        #&  (sf.col("FIRST_NAME").rlike(name_regex))
        #&  (sf.col("LAST_NAME").rlike(name_regex))
        ))
    elif type == "encounters":
        df = (df.where(
            (sf.col("DATE").isNotNull())
        &   (sf.col("MRN").isNotNull())
        &   (sf.col("ICD_CODE").isNotNull())
        &   (sf.col("DATE").rlike(date_regex))
        &   (sf.col("DATE") > sf.date_sub(sf.current_date(), (15 * 365)))
        &   (sf.col("ICD_CODE").rlike(icd9_regex) | sf.col("ICD_CODE").rlike(icd10_regex))
        ))

    df = df.dropDuplicates()

    return df

code_groups_sdf = clean_data(code_groups_sdf, "code_groups")
demo_sdf = clean_data(demo_sdf, "demographics")
enc_sdf = clean_data(enc_sdf, "encounters")


In [801]:
run_all_checks()

-----CODE GROUPS-----


+---------------+
|Duplicate Count|
+---------------+
|              0|
+---------------+

Null Counts
+--------+-----+
|ICD_CODE|GROUP|
+--------+-----+
|       0|    0|
+--------+-----+



+-------+--------+------------------+
|summary|ICD_CODE|             GROUP|
+-------+--------+------------------+
|  count|    4995|              4995|
|   mean|    null| 5.531131131131131|
| stddev|    null|2.8903991264796374|
|    min|   A07.0|                 1|
|    max| Z98.818|                 9|
+-------+--------+------------------+

-----DEMO-----


+---------------+
|Duplicate Count|
+---------------+
|              0|
+---------------+

Null Counts


+------+---+----------+---------+-------------+
|CLINIC|MRN|FIRST_NAME|LAST_NAME|DATE_OF_BIRTH|
+------+---+----------+---------+-------------+
|     0|  0|         0|        0|            0|
+------+---+----------+---------+-------------+



+-------+------+--------------------+----------+---------+
|summary|CLINIC|                 MRN|FIRST_NAME|LAST_NAME|
+-------+------+--------------------+----------+---------+
|  count|  9999|                9999|      9999|     9999|
|   mean|  null| 5.075196392889289E7|      null|     null|
| stddev|  null|2.8727837480940565E7|      null|     null|
|    min|     A|            01015674|     Aaron|   Abbott|
|    max|     C|            99998883|       Zoe|   Zuniga|
+-------+------+--------------------+----------+---------+

-----ENC-----


+---------------+
|Duplicate Count|
+---------------+
|              0|
+---------------+

Null Counts


+----+------+---+--------+
|DATE|ENC_ID|MRN|ICD_CODE|
+----+------+---+--------+
|   0|     0|  0|       0|
+----+------+---+--------+



+-------+--------------------+---------+--------+
|summary|              ENC_ID|      MRN|ICD_CODE|
+-------+--------------------+---------+--------+
|  count|              131445|   131445|  131445|
|   mean| 4.986884302184183E7|     null|    null|
| stddev|2.8775586191770155E7|     null|    null|
|    min|            10000905|A01126333|   A07.0|
|    max|            99999335|C99975199| Z98.818|
+-------+--------------------+---------+--------+



In [802]:
def transform_encounters(enc_sdf):
    sdf2 = (
        enc_sdf
        .withColumn('ROW_ID', sf.monotonically_increasing_id())
        .withColumn('PRIORITY', sf.row_number().over(Window.partitionBy('ENC_ID').orderBy('ROW_ID')))
        .drop('ROW_ID')
        .groupBy(['ENC_ID', 'DATE', 'MRN'])
        .pivot('PRIORITY')
        .agg(sf.first('ICD_CODE'))
    )
    return rename_columns(sdf2, {i: f'ICD_CODE_{i}' for i in sdf2.columns[3:]})

In [803]:
enc_sdf = transform_encounters(enc_sdf)

In [804]:
enc_sdf.show()

+--------+----------+---------+----------+----------+----------+----------+
|  ENC_ID|      DATE|      MRN|ICD_CODE_1|ICD_CODE_2|ICD_CODE_3|ICD_CODE_4|
+--------+----------+---------+----------+----------+----------+----------+
|10106857|2020-11-26|A22804925|  S81.831S|     Z33.1|      null|      null|
| 1033030|2020-09-14|B24619794|       N06|  S52.344J|     F11.9|  S90.00XA|
| 1111477|2017-07-08|B85149257|   S52.514|  S81.049D|  S72.466Q|      null|
|11146680|2021-01-29|A02882218|     B00.1|  S62.637A|   T48.995|     S16.8|
|11644186|2020-07-16|B95805179|  S02.11FK|  S68.619S|       K70|  S42.353B|
|11798180|2019-10-09|A66060669|  S32.16XA|  S90.464S|  S20.309D|      null|
|12498017|2020-07-31|C51507666|  S58.029D|      null|      null|      null|
| 1317551|2020-01-01|C36980007|  S92.052S|      null|      null|      null|
|13508582|2019-05-16|B49278302|  S45.191D|  S74.01XD|      null|      null|
|13613552|2019-12-11|A40618071|   I83.002|    C40.20|  S83.105S|      null|
|14048899|20

In [805]:
demo_sdf = demo_sdf.withColumn("FULL_MRN", sf.concat("CLINIC", "MRN"))

In [806]:
enc_sdf_added_cols = (
    enc_sdf
    .join(demo_sdf, enc_sdf.MRN == demo_sdf.FULL_MRN)
    .withColumn('AGE', sf.datediff('DATE', 'DATE_OF_BIRTH') / 365)
    .withColumn('AGE_BUCKET', sf.udf(age_bucket)(sf.col('AGE')))
    .withColumn('ICD_CODE', sf.expr("stack(4, ICD_CODE_1, ICD_CODE_2, ICD_CODE_3, ICD_CODE_4)"))
)
results = (
    enc_sdf_added_cols
    .join(code_groups_sdf, enc_sdf_added_cols.ICD_CODE==code_groups_sdf.ICD_CODE)
    .groupBy('AGE_BUCKET', 'GROUP').count()
    .groupBy('AGE_BUCKET', 'count').agg(sf.collect_set('GROUP'))
    .withColumnRenamed('collect_set(GROUP)', 'GROUPS')
    .withColumn(
        '_row',
        sf.row_number().over(Window().partitionBy(['AGE_BUCKET']).orderBy(sf.desc('count'))))
    .filter(sf.col('_row') == 1)
    .drop('_row')
    .filter(sf.col('AGE_BUCKET') != 0)
    .orderBy('AGE_BUCKET')
)

In [807]:
results.show()

+----------+-----+------+
|AGE_BUCKET|count|GROUPS|
+----------+-----+------+
|       100| 1448|  [10]|
|        40| 1087|  [10]|
|        50| 1222|  [10]|
|        60| 1139|  [10]|
|        70| 1066|  [10]|
|        80| 1038|  [10]|
|        90| 1167|  [10]|
+----------+-----+------+

